In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
import sqlite3
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

from dotenv import load_dotenv
load_dotenv()

True

In [5]:
# Initialize the Gemini model via Vertex AI
vertex_llm = init_chat_model(
    "gemini-2.5-flash", 
    model_provider="google_vertexai",
    project='gen-lang-client-0311515393',
    location='us-central1',
)

type(vertex_llm)


/Users/sawale/Documents/learning/money_rag/.venv/lib/python3.12/site-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


langchain_google_vertexai.chat_models.ChatVertexAI

In [4]:

openai_llm = init_chat_model(
    "gpt-4o",  # or "gpt-3.5-turbo", "o1-preview", etc.
    model_provider="openai",
    api_key="your-api-key-here", # Optional: can also be set via OPENAI_API_KEY env var
)

type(openai_llm)

langchain_openai.chat_models.base.ChatOpenAI

In [7]:
from langchain.agents import create_agent

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_agent(
    # model="gpt-4o",
    model=vertex_llm,
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='6eec9126-cda3-4cdd-9f6d-f893b228c7cc'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city": "sf"}'}, '__gemini_function_call_thought_signatures__': {'28adfdf3-778e-4d93-bd45-94a38f64064f': 'CtsFAY89a1+qAQDcM0PFB0w9LKN21xyUOLVEzccWGHcyKXyKAye7ST4azyzp32wcQLIbbiA4N5NSZO5Ua07pNp6Ey+dHG7MoVMoirXyDabR/IBeaesEOIg3fSaXAegxgzEjgLEmmADwAJRxvGpTNyalfZDUPkh0UcRcwGB6jqibG261Euk9G3hkXrHNBig5gBG1uoAxDxwrydV3T57wFRIxn4EEU+IqDiQRdMFsVn8ANZ4BzcpfRdNKsE/VRJ/htqEAHGeEk3jlqZRcJmkRCFHQvsRMxMI911jBH1puimK61Ah3r6P0aMtfkh4wqAENd+Oou06CG5Iz8IS1kBXNxL2/XbqdCDiSmIepKZNVmkyShmt315nPsQ1U3uM7N6415goYjW0q3IouuhiTl1Q6xwzrCfITXgBa9TL09x/CcSOIW9LgRJ6z7PX20PByOZ/rcqsdxr3lGgGSv+3P4b22mDPrrXZEHs8Hkw+1Fw4QMmFJLcj6WCXJaxlRaNR9TMlcspJGT+AtSv1X287cKqyyKGL6x536j5/vU4xDf2do4QIN/Rom0Xj7d46vUS9/B5abHSvfIErAlwDLcGDvHE3S2nC8gN4GKuTSDSm88DLhv46UwLov/NrF/6v9n7ZiqLkvHl/B

In [6]:
from typing import TypedDict
from dataclasses import dataclass
from langgraph.runtime import get_runtime
from langgraph.checkpoint.memory import InMemorySaver
from langchain_community.utilities import SQLDatabase
from langchain_core.tools import tool
from langchain.agents import create_agent


db = SQLDatabase.from_uri("sqlite:///money_rag.db")

@dataclass
class RuntimeContext:
    db: SQLDatabase

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db
    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

SYSTEM = f"""You are a careful SQLite analyst.
Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows of output unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
- **Spending Logic**:
    - Spending is represented by POSITIVE values (> 0) in the amount column.
    - NEGATIVE values (< 0) represent credit card payments or refunds and MUST NOT be included when calculating total spending or expenses.
"""
agent = create_agent(
 model=vertex_llm,
 tools=[execute_sql],
 system_prompt=SYSTEM,
 context_schema=RuntimeContext,
 checkpointer=InMemorySaver(),
)

In [7]:
question = "how much total amount i spent for movie ticket in last 6 months?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    stream_mode="values",
    context=RuntimeContext(db=db)
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

how much total amount i spent for movie ticket in last 6 months?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (13c234ed-8b6e-4db2-b4cd-ebff7fa65c1b)
 Call ID: 13c234ed-8b6e-4db2-b4cd-ebff7fa65c1b
  Args:
    query: SELECT SUM(amount) FROM transactions WHERE category = 'movie ticket' AND date >= strftime('%Y-%m-%d', date('now', '-6 months')) AND amount > 0
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such column: date
[SQL: SELECT SUM(amount) FROM transactions WHERE category = 'movie ticket' AND date >= strftime('%Y-%m-%d', date('now', '-6 months')) AND amount > 0]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================
Tool Calls:
  execute_sql (3b514cf6-71cd-4730-b7d1-

In [21]:
question = "what transcation did you take into account for this calculation?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    stream_mode="values",
    context=RuntimeContext(db=db)
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

what transcation did you take into account for this calculation?
================================== Ai Message ==================================

I need more information to answer your question. Could you please tell me which calculation you are referring to? I don't have memory of past interactions or calculations.
